In [8]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import scipy as sc
import scipy.spatial.distance as ssd
from scipy.cluster import hierarchy
import matplotlib.pyplot as plt

In [9]:
def reading(read_path,save_path=None):

    df = pd.read_csv(read_path, sep=",")

    return df

df = reading("/Users/roicort/Documents/GitHub/DarkMagic/NLP/corpus.csv")

In [10]:
df

,Id,Título,Oración,Objeto,Dónde se compró,Medidas,Datos adicionales,Voz,Otras versiones
0,1,Legítimo polvo Sándalo,NaN,Polvo blanco,"Mercado de Catemaco, Veracruz (2008)",9 x 6 cm,Instrucciones: “Acostumbre usar diariamente es...,NaN,NaN
1,2,MARAVILLOSO POLVO DEL CONGO,NaN,Polvo blanco,Mercado de Catemaco (2008),9 x 6 cm,"“RETIRA SALACIONES EN EL NEGOCIO, TRABAJO Y HO...",NaN,NaN
2,3,LEGÍTIMO POLVO DOBLEGADO A MIS PIES [ver #51],NaN,Polvo blanco,Mercado de Catemaco (2008),9 x 6 cm,"“Este polvo tiene propiedades ocultas, la muje...","mujer, para la",NaN
3,4,POLVO DE OLICORNIO VFNCEDOR [sic],NaN,Polvo gris claro,Mercado de Catemaco (2008),9 x 6 cm,“Legítimo Polvo VENCEDOR úselo polveándose”,NaN,NaN
4,5,Polvo especial SIEMPRE CONMIGO,NaN,Polvo blanco,Mercado de Catemaco (2008),9 x 6 cm,USELO POLVEANDOSE TODO EL CUERPO,NaN,NaN
...,...,...,...,...,...,...,...,...,...
140,140,LEGITIMO POLVO DE Llama Cliente,NaN,Polvo gris claro,Mercado Independencia de Morelia (2016),NaN,LEGITIMO POLVO DE LLAMA CLIENTE [ilegible],NaN,NaN
141,141,MARAVILLOSO POLVO CONGO,NaN,Polvo gris,Mercado Independencia de Morelia (2016),NaN,"RETIRA SALACIONES EN EL NEGOCIO, TRABAJO Y HOG...",NaN,NaN
142,142,Polvo Divina Providencia,A ti excelsa trinidad mi memoria y voluntad a ...,Polvo gris claro,NaN,NaN,NaN,NaN,NaN
143,143,LEGITIMO POLVO DE LAS CUATRO MAGIAS,NaN,Polvo gris claro,NaN,NaN,LEGITIMO POLVO DE LAS CUATRO MAGIAS Líbrame ...,NaN,NaN


In [2]:
set(df["Dónde se compró"])

{'(2017) Mariana',
 'Mercado De Catemaco (2017)',
 'Mercado Independencia  de Morelia (2016)',
 'Mercado Independencia Morelia (2017)',
 'Mercado Independencia de Morelia',
 'Mercado Independencia de Morelia (2015)',
 'Mercado Independencia, Morelia',
 'Mercado Independiencia, Morelia',
 'Mercado de Catemaco',
 'Mercado de Catemaco (2008)',
 'Mercado de Catemaco (2015)',
 'Mercado de Catemaco (2016)',
 'Mercado de Catemaco (2017)',
 'Mercado de Catemaco, Veracruz (2008)',
 'Mercado de Jamaica, México (puesto 1)',
 'Mercado de Jamaica, México (puesto 1)\n(32 bis: puesto 2)',
 'Mercado de Jamaica, México (puesto 2)',
 'Mercado de Jamaica, México, (puesto 1)',
 'Mercado de Jamaicoa, Mexico (Puesto 1)',
 'Mercado de Sonora',
 'Mercado de Sonora, México (2015)',
 'Mercado independencia, Morelia',
 'Para mayor resultado y tener suerte use este amuleto de doble suerte',
 nan,
 '¿Pátzcuaro? (Mariana)',
 '¿Pátzcuaro? (Mariana) (2015)',
 '¿Pátzcuaro? (Mariana) (2015). Mercado Independencia (2017

In [3]:
set(df["Voz"])

{' femenina',
 'Masculina',
 'No venía el polvo! ',
 'hombre o mujer',
 'inditintaq',
 'mujer',
 'mujer, para la ',
 nan}

In [4]:
import spacy
from collections import Counter
from string import punctuation
from spacy import displacy

In [12]:
nlp = spacy.load("es_core_news_sm")
text = df.iloc[7]["Oración"].replace("\n"," ")
print(text)
# Process whole documents
doc = nlp(text)
print("\n")
# Analyze syntax
print("Nouns:", [chunk.text for chunk in doc.noun_chunks])
print("\n")
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])
print("\n")
# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

ORACION A SAN ALEJO San Alejo, San Alejo tras veces te he de llamar cuantas veces se me ofrezca librame de todo mal. Estas tres cruces que hago, que hago en señal del buen cristiano del criminal o villano para que castigues la mano que quiera hacerme mal, así también quebrarás la lengua del que hable mal de mi. Te ruego glorioso San Alejo no abandones los alrededor [sic] de mi casa ni lo que esté al pie de mi obligación San Alejo de León, mis enemigos del mundo del mar y del monte sean, las personas  que quieran traicionarme por mi corazón le caigan las alas del corazón y venga rendido a mi como Jesús en la Cruz. Amen, Jesús, San Alejo, San Alejo A nuestros malos vecinos retiralos lejos, lejos


Nouns: ['SAN ALEJO San Alejo', 'San Alejo', 'veces', 'te', 'veces', 'se', 'me', 'todo', 'Estas tres cruces', 'cristiano', 'criminal', 'la mano', 'la lengua', 'Te', 'ruego', 'San Alejo', 'abandones', ']', 'mi casa', 'lo', 'lo que esté al pie de mi obligación', 'San Alejo de León', 'mis enemigos'

In [6]:
def get_key(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] # 1
    doc = nlp(text.lower()) # 2
    for token in doc:
        # 3
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        # 4
        if(token.pos_ in pos_tag):
            result.append(token.text)
    print("Keys Extracted")
    return list(set(result)) # 5

In [7]:
get_key(text)

Keys Extracted


['abandones',
 'villano',
 'personas',
 'león',
 'oracion',
 'monte',
 'cristiano',
 'alejo',
 'retiralos',
 'amen',
 'lengua',
 'enemigos',
 'alas',
 'glorioso',
 'mar',
 'malos',
 'casa',
 'jesús',
 'san',
 'rendido',
 'vecinos',
 'mundo',
 'cruces',
 'corazón',
 'criminal',
 'mano',
 'librame',
 'obligación',
 'cruz']

In [60]:
allverbs=[]
for index, row in df.iterrows():
    text = str(row["Oración"]).replace("\n"," ").lower()
    doc = nlp(text)
    verbs = [token.lemma_ for token in doc if token.pos_ == "VERB"]
    if verbs != []:
        for verb in verbs:
            allverbs.append(verb.replace())

In [66]:
from collections import Counter
d = dict(Counter(allverbs))

In [67]:
sort = sorted(d.items(), key=lambda x: x[1], reverse=True)

In [68]:
sort

[('ser', 67),
 ('pedir', 48),
 ('tener', 47),
 ('venir', 36),
 ('querer', 29),
 ('[', 27),
 ('llamar', 25),
 ('estar', 21),
 ('poder', 20),
 ('hacer', 18),
 ('dar', 18),
 ('dominar', 15),
 ('conceder', 15),
 ('sentir', 13),
 ('vencer', 13),
 ('animar', 13),
 ('ver', 11),
 ('volver', 11),
 ('poner', 10),
 ('decir', 10),
 ('ofrecer', 9),
 ('necesitar', 9),
 ('dormir', 9),
 ('traer', 9),
 ('desear', 9),
 ('ir', 8),
 ('alejar', 8),
 ('amar', 8),
 ('mirar', 7),
 ('comer', 7),
 ('encontrar', 6),
 (']', 6),
 ('retirar', 6),
 ('llegar', 6),
 ('permitir', 6),
 ('hablar', 5),
 ('abrir', 5),
 ('arrodillar', 5),
 ('quedar', 5),
 ('llevar', 5),
 ('cumplir', 5),
 ('caer', 4),
 ('repetir', 4),
 ('pasar', 4),
 ('dejar', 4),
 ('enamorar', 4),
 ('meter', 4),
 ('implorar', 4),
 ('olvidar', 4),
 ('ganar', 4),
 ('invocar', 4),
 ('haber', 4),
 ('castigar', 3),
 ('buscar', 3),
 ('detener', 3),
 ('soñar', 3),
 ('llorar', 3),
 ('perdonar', 3),
 ('conjurar', 3),
 ('rendir', 3),
 ('acompañar', 3),
 ('salir', 3),